In [1]:
from main import *
from sklearn.model_selection import train_test_split

import keras
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.utils import shuffle

from tensorflow.keras import layers, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.utils import to_categorical
from itertools import groupby
from sklearn.preprocessing import MinMaxScaler


SHORT = True

2024-11-09 12:33:10.702377: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-09 12:33:10.755110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731151990.782976    6672 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731151990.790527    6672 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 12:33:10.839446: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
if SHORT == True:
    machines = ["M01", "M02", "M03"]
    process_names = ["OP01", "OP02","OP07"]
else:
    machines = ["M01", "M02","M03"]
    process_names = ["OP00","OP01","OP02","OP03","OP04","OP05","OP06","OP07","OP08","OP09","OP10","OP11","OP12","OP13","OP14"]

labels = ["good","bad"]
path_to_dataset = Path("./data/").absolute()

In [3]:
def resize_op(op):
    min = float("inf")
    for x in op:
        if x.shape[0] < min:
            min = x.shape[0]
    for i in range(len(op)):
        op[i] = op[i][:min]
    return op

def group_ops(op):
    keyfunc = lambda x: x[4:12]
    groups = []
    uniquekeys = []
    data = sorted(op, key=keyfunc)
    for k, g in groupby(data, keyfunc):
        groups.append(list(g))      # Store group iterator as a list
        uniquekeys.append(k)
    return groups

def select_samples(Y0, Y1, Y2, X0, X1, X2):
    X, y = [], []
    
    idx0, idx1, idx2 = 0, 0, 0
    count = 0
    for y0, y1, y2 in zip(Y0, Y1, Y2):
        len0 = len(y0)
        len1 = len(y1)
        len2 = len(y2)
        minimum = min(len0, len1, len2)
        for i in range(minimum):
            X.append(np.concatenate((X0[idx0 + i], X1[idx1 + i], X2[idx2 + i])))
            y.append((1 if any("bad" in s for s in [y0[i], y1[i], y2[i]]) else 0))
            count+= (1 if "bad" in (y0[i][-4:] + y1[i][-4:] + y2[i][-4:]) else 0)
        idx0 += len0 
        idx1 += len1
        idx2 += len2
    return np.array(X), np.array(y)

def split_samples(X, y):
    all_indices = np.arange(len(y))
    normal = np.where(np.array(y) == 0)[0]
    sample_size = int(len(y) * 0.6)
    train_indices = np.random.choice(normal, size=sample_size, replace=False)
    testval_indices = np.setdiff1d(all_indices, train_indices)
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_testval = X[testval_indices]
    y_testval = y[testval_indices]
    
    X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, test_size = 0.75, stratify = y_testval)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [4]:
OP01_X_data, OP01_y_data = [], []
OP02_X_data, OP02_y_data  = [], []
OP07_X_data, OP07_y_data = [], []

for process_name, machine, label in itertools.product(process_names, machines, labels):
    data_path = os.path.join(path_to_dataset, machine, process_name, label)
    data_list, data_label = data_loader_utils.load_tool_research_data(data_path, label=label, add_additional_label = True, verbose = False)
    if process_name == "OP01":
        OP01_X_data.extend(data_list)
        OP01_y_data.extend(data_label)
    if process_name == "OP02":
        OP02_X_data.extend(data_list)
        OP02_y_data.extend(data_label)
    if process_name == "OP07":
        OP07_X_data.extend(data_list)
        OP07_y_data.extend(data_label)

X0 = resize_op(OP01_X_data)
X1 = resize_op(OP02_X_data)
X2 = resize_op(OP07_X_data)

Y0 = group_ops(OP01_y_data)
Y1 = group_ops(OP02_y_data)
Y2 = group_ops(OP07_y_data)

X, y = select_samples(Y0, Y1, Y2, X0, X1, X2)

X_train, X_val, X_test, y_train, y_val, y_test = split_samples(X, y)

In [5]:
def compute_scalogram(x):
    x_scalogram_list = []
    for m in range(3):
        cwtmatr, freqs = cwt(x[:, m], np.arange(1, 129), "morl")
        cwtmatr_reshaped = resize(cwtmatr, (128, 512))
        x_scalogram_list.append(cwtmatr_reshaped)
    x_scalogram_raw = np.array(x_scalogram_list)
    x_scalogram = x_scalogram_raw.reshape(128, 512, 3)
    return x_scalogram

def augment(T, X_data, p):
    X = np.copy(X_data)
    X_augmented_list = []
    n_samples, n_timesteps, n_sensors = X.shape

    loc = []
    trans = []
    
    b = compute_windows(n_timesteps, p) # Compute window bounds for p windows

    for i, t in enumerate(T):
        print(f"Processing {t.__name__} transformations")
        for x in X:
            W_j = np.random.randint(1, p + 1)  # Randomly choose a window W_j
            b_lower, b_upper = b[W_j]  # Get the upper and lower bound of the windows
            c1, c2 = sorted(np.random.randint(b_lower, b_upper + 1, size=2))  # Sample two random numbers within the window
            if c1 > c2:
                c1, c2 = c2, c1
            x_augmented = t(x, c1, c2, p, b)  # Augment x in window W_j with t
            x_scalogram = compute_scalogram(x_augmented)
            X_augmented_list.append(x_scalogram)
            
            if i == 0:
                loc.append(0)
            else:
                loc.append(W_j)
            trans.append(i)
        print(f"Completed Processing {t.__name__} transformations") 

    
    return np.array(X_augmented_list), np.array(loc), np.array(trans)

def normalize_axis(X_data, X_train):
    X = np.copy(X_data)
    for i in range(3):
        min_val = np.min([np.min(x) for x in X_train[:,:,:,i]])
        max_val = np.max([np.max(x) for x in X_train[:,:,:,i]])
    
        X[:,:,:,i] = [(x - min_val) / (max_val - min_val) for x in X[:,:,:,i]]
    return X

In [6]:
T = [identity, cut_paste, mean_shift, missing_signal]

X_aug, loc, trans  = augment(T, X_train, 5)

Processing identity transformations
Completed Processing identity transformations
Processing cut_paste transformations
Completed Processing cut_paste transformations
Processing mean_shift transformations
Completed Processing mean_shift transformations
Processing missing_signal transformations
Completed Processing missing_signal transformations


In [7]:
X_aug_n = normalize_axis(X_aug, X_aug)
X_aug_n, loc, trans = shuffle(X_aug_n, loc, trans)

loc = to_categorical(loc, num_classes=6).reshape(-1, 6)
trans = to_categorical(trans, num_classes=4).reshape(-1, 4)

X_aug_list = list(X_aug_n)
loc_list = list(loc)
trans_list = list(trans)

train_aug = tf.data.Dataset.from_tensor_slices((X_aug_list, (loc_list, trans_list)))
train_aug = train_aug.batch(8)

I0000 00:00:1731153232.333650    6672 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
slope = 1e-1
input_shape = (128, 512, 3)

sensordata = keras.Input(shape = input_shape, name = "sensordata")

x = layers.Conv2D(8, (7, 7))(sensordata)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(slope)(x)

x = layers.MaxPooling2D()(x)

x = layers.Conv2D(16, (7, 7))(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(slope)(x)

x = layers.MaxPooling2D()(x)

x = layers.Conv2D(32, (7, 7))(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(slope)(x)

x = layers.MaxPooling2D()(x)

x = layers.Flatten()(x)
x = layers.Dense(32)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(slope)(x)

x = layers.Dense(16)(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(slope)(x)

g_loc = layers.Dense(16)(x)
g_loc = layers.BatchNormalization()(g_loc)
g_loc = layers.LeakyReLU(slope)(g_loc)

g_loc = layers.Dense(8)(g_loc)
g_loc = layers.BatchNormalization()(g_loc)
g_loc = layers.LeakyReLU(slope)(g_loc)
loc = layers.Dense(6, activation = "softmax")(g_loc)

g_trans = layers.Dense(16)(x)
g_trans = layers.BatchNormalization()(g_trans)
g_trans = layers.LeakyReLU(slope)(g_trans)

g_trans = layers.Dense(8)(g_trans)
g_trans = layers.BatchNormalization()(g_trans)
g_trans = layers.LeakyReLU(slope)(g_trans)
trans = layers.Dense(4, activation = "softmax")(g_trans)

model = keras.Model(inputs = sensordata, outputs =[loc, trans])

In [9]:
initial_learning_rate = 1e-4
steps = (340 // 8) * 20
lr_schedule = CosineDecay(initial_learning_rate, steps)

model.compile(optimizer=Adam(lr_schedule, weight_decay = 1e-3),
              loss = ['categorical_crossentropy', 'categorical_crossentropy'],
              metrics = [["accuracy"], ["accuracy"]])

In [10]:
model.fit(train_aug,
          epochs = 20)

Epoch 1/20


I0000 00:00:1731153255.359397    6884 service.cc:148] XLA service 0x7f90300244a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731153255.359533    6884 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-11-09 12:54:15.505687: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731153255.976531    6884 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1731153261.283549    6884 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 ━━━━━━━━━━━━━━━━━━━━ 16s 142ms/step - dense_4_accuracy: 0.1771 - dense_4_loss: 1.9395 - dense_7_accuracy: 0.2351 - dense_7_loss: 1.6098 - loss: 3.5500
Epoch 2/20


2024-11-09 12:54:27.238407: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion', 8 bytes spill stores, 8 bytes spill loads



43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - dense_4_accuracy: 0.1810 - dense_4_loss: 1.8603 - dense_7_accuracy: 0.2837 - dense_7_loss: 1.4171 - loss: 3.2776
Epoch 3/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - dense_4_accuracy: 0.2766 - dense_4_loss: 1.7355 - dense_7_accuracy: 0.3622 - dense_7_loss: 1.3089 - loss: 3.0447
Epoch 4/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - dense_4_accuracy: 0.3068 - dense_4_loss: 1.6731 - dense_7_accuracy: 0.4436 - dense_7_loss: 1.2085 - loss: 2.8819
Epoch 5/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - dense_4_accuracy: 0.3249 - dense_4_loss: 1.6068 - dense_7_accuracy: 0.4753 - dense_7_loss: 1.1702 - loss: 2.7773
Epoch 6/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - dense_4_accuracy: 0.3607 - dense_4_loss: 1.5722 - dense_7_accuracy: 0.4727 - dense_7_loss: 1.1360 - loss: 2.7084
Epoch 7/20
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - dense_4_accuracy: 0.3784 - dense_4_loss: 1.5324 - dense_7_accuracy: 0.5340 - dense_7_loss: 1.1026 - loss: 2.6351
Epoch 8/20
43/4

In [11]:
dst = keras.Model(inputs = sensordata, outputs = x, trainable = False)

In [12]:
dst.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sensordata (InputLayer)         │ (None, 128, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 122, 506, 8)    │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 122, 506, 8)    │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 122, 506, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 61, 253, 8)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 55, 247, 16)    │         6,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 55, 247, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 55, 247, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 27, 123, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 117, 32)    │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 21, 117, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 21, 117, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 58, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18560)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       593,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 16)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 627,488 (2.39 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 627,488 (2.39 MB)

In [13]:
X_train_scal = np.array(list(map(compute_scalogram, X_train)))
X_train_normal = normalize_axis(X_train_scal, X_train_scal)

X_val_scal = np.array(list(map(compute_scalogram, X_val)))
X_val_normal = normalize_axis(X_val_scal, X_train_scal)

#X_test_scal = np.array(list(map(compute_scalogram, X_test)))
#X_test_normal = normalize_axis(X_test_scal, X_train_scal)))

In [14]:
output = dst.predict(X_train_normal)

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 566ms/step


In [22]:
from scipy.stats import gaussian_kde

mean = np.mean(output, axis = 0)
cov = np.cov(output.T)
inv_cov = np.linalg.inv(cov)

t2_scores = []
for t2 in output:
    t2_scores.append(float((t2 - mean).T @ inv_cov @ (t2 - mean)))

t2_scores = np.array(t2_scores)

def K(x):
    return np.exp(-x**2/2)/np.sqrt(2*np.pi)

h = 3
totalsum = 0
for t2 in t2_scores:
    totalsum += K((t2_scores - t2) /h)


phi = (1 / (85 * 3)) * totalsum

In [23]:
ucl = np.quantile(phi, 0.99)

In [24]:
val_output = dst.predict(X_val_normal)
t2_scores_val = []
for x in val_output:
    t2_scores_val.append(float((x - mean).T @ inv_cov @ (x - mean)))
t2_scores_val = np.array(t2_scores_val)
preds = []

h = 3
totalsumval = 0
for t2 in t2_scores_val:
    totalsumval += K((t2_scores_val - t2) /h)
phi_val = (1 / (14 * 3)) * totalsumval

for score in phi_val:
    if score> ucl:
        preds.append(1)
    else:
        preds.append(0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [25]:
preds

[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1]

In [26]:
y_val

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])